In [1]:
# import numpy as np
import pandas as pd
import math
import os
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 200)

In [2]:
# context = pd.read_csv('..//data/preprocessed//context_1217_1230.csv')
context = pd.read_csv('/home/jovyan/df-smart-channel/graph/data/preprocessed/context_1217_0131.csv')

In [3]:
le = LabelEncoder()
context.cust_no = le.fit_transform(context.cust_no)
context.item_id = le.fit_transform(context.item_id)

In [4]:
user_list = context.cust_no.unique()
item_list = context.item_id.unique()

## Splitting

In [5]:
train_start = 20211217
train_end = 20211225
test_end = 20211226

In [6]:
def splitting(user_list, train_start, test_date):
    context_train = context[(context.date >= train_start) & (context.date < test_date)][['cust_no', 'item_id', 'show', 'click']]
    context_test = context[context.date == test_date][['cust_no', 'item_id', 'show', 'click']]
    
    cust_item_pair_train = context_train.groupby(['cust_no', 'item_id'])['click'].sum().reset_index()
    cust_item_pair_test = context_test.groupby(['cust_no', 'item_id'])['click'].sum().reset_index()
    
    cust_item_pair_train = cust_item_pair_train[cust_item_pair_train.click > 0]
    cust_item_pair_test = cust_item_pair_test[cust_item_pair_test.click > 0]
    # init_df = pd.DataFrame({"cust_no": range(max(user_list)+1)})
    # ## mapping
    # cust_item_pair_train = pd.merge(init_df, cust_item_pair_train, on = 'cust_no', how = 'left')
    # cust_item_pair_test = pd.merge(init_df, cust_item_pair_test, on = 'cust_no', how = 'left')
    
    cust_item_pair_train.click = cust_item_pair_train.click.fillna(0)
    cust_item_pair_test.click = cust_item_pair_train.click.fillna(0)
    
    items_by_cust_train = cust_item_pair_train.groupby('cust_no')['item_id'].unique()
    items_by_cust_test = cust_item_pair_test.groupby('cust_no')['item_id'].unique()
    
    cust_items_dict_train = dict(items_by_cust_train.apply(list))
    cust_items_dict_test = dict(items_by_cust_test.apply(list))
    return cust_items_dict_train, cust_items_dict_test

In [8]:
def export_file(date, pairs_dict_train, pairs_dict_test):
    dir_name = f'date={date}'
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
        
    ## train
    file_path = os.path.join(dir_name, 'train.txt')
    file = open(file_path,"w")

    for key,value in pairs_dict_train.items():
        file.write('%s' % key)
        # for item in value:
        #     file.write('%s ' % item)
        for i, item in enumerate(value):
            if math.isnan(item):
                continue
            else:
                if i == (len(value)-1):
                    file.write(' %s' % int(item))
                else:
                    file.write(' %s' % int(item))
        file.write('\n')
    file.close()
    ## test
    file_path = os.path.join(dir_name, 'test.txt')
    file = open(file_path,"w")

    for key,value in pairs_dict_test.items():
        file.write('%s' % key)
        # for item in value:
        #     file.write('%s ' % item)
        for i, item in enumerate(value):
            if math.isnan(item):
                continue
            else:
                if i == (len(value)-1):
                    file.write(' %s' % int(item))
                else:
                    file.write(' %s' % int(item))
        file.write('\n')
    file.close()

In [11]:
for test_date in range(20211226, 20211231+1):
    cust_items_dict_train, cust_items_dict_test = splitting(user_list = user_list, train_start = 20211217, test_date = test_date) 
    export_file(date = str(test_date), pairs_dict_train = cust_items_dict_train, pairs_dict_test = cust_items_dict_test )